In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm



# <font color='red'>训练集缺陷代码行提取

In [2]:
train_df = pd.read_pickle('../../data/jitfine/features_train.pkl')
# buggy_changes_with_buggy_line.json中知识对buggy commit的标注
train_df = train_df[train_df['is_buggy_commit']==1]
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1390 entries, 0 to 16065
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   project                     1390 non-null   object 
 1   parent_hashes               1390 non-null   object 
 2   commit_hash                 1390 non-null   object 
 3   author_name                 1390 non-null   object 
 4   author_email                1390 non-null   object 
 5   author_date                 1390 non-null   object 
 6   author_date_unix_timestamp  1390 non-null   object 
 7   commit_message              1390 non-null   object 
 8   la                          1390 non-null   object 
 9   ld                          1390 non-null   object 
 10  fileschanged                1390 non-null   object 
 11  nf                          1390 non-null   object 
 12  ns                          1390 non-null   object 
 13  nd                          1390

In [3]:
path = '../../JITFine/labels for each line/buggy_changes_with_buggy_line.json'
data = open(path, encoding='utf-8')
data = json.load(data)
# data.keys()
prj_list = list(data.keys())
print(len(prj_list))
prj_list

26


['ant-ivy',
 'commons-math',
 'opennlp',
 'parquet-mr',
 'archiva',
 'deltaspike',
 'systemml',
 'commons-lang',
 'commons-net',
 'commons-collections',
 'commons-beanutils',
 'commons-codec',
 'commons-compress',
 'commons-configuration',
 'commons-digester',
 'commons-jcs',
 'commons-io',
 'commons-scxml',
 'commons-validator',
 'commons-vfs',
 'giraph',
 'jspwiki',
 'eagle',
 'commons-bcel',
 'commons-dbcp',
 'gora']

In [4]:
not_use_prj = ['archiva', 'deltaspike', 'systemml', 'jspwiki',
               'eagle']
for k in not_use_prj:
    del data[k]
prj_list = list(data.keys())
print(len(prj_list))
prj_list

21


['ant-ivy',
 'commons-math',
 'opennlp',
 'parquet-mr',
 'commons-lang',
 'commons-net',
 'commons-collections',
 'commons-beanutils',
 'commons-codec',
 'commons-compress',
 'commons-configuration',
 'commons-digester',
 'commons-jcs',
 'commons-io',
 'commons-scxml',
 'commons-validator',
 'commons-vfs',
 'giraph',
 'commons-bcel',
 'commons-dbcp',
 'gora']

In [5]:
commit_lines_info = []

for i in tqdm(range(len(train_df))):
    commit = train_df.iloc[i]
    prj = commit['project']
    # print(prj)
    commit_hash = commit['commit_hash']


    info = data[prj][commit_hash]['added_buggy_level']
    modified_files = list(info.keys())



    idx = 0
    for file_name in modified_files:
        file = info[file_name]

        if 'added_buggy' in file.keys():
            buggy_lines = file['added_buggy']

            for line in buggy_lines:
                if line=='':
                    print('oh my god')
                temp = [prj, file_name, 'added', commit_hash, idx, line, 1]
                commit_lines_info.append(temp)
                idx += 1


        if 'added_clean' in file.keys():
            clean_lines = file['added_clean']

            for line in clean_lines:
                if line=='':
                    print('oh my god')
                temp = [prj, file_name, 'added', commit_hash, idx, line, 0]
                commit_lines_info.append(temp)
                idx += 1

        if 'deleted' in file.keys():
            deleted_lines = file['deleted']

            for line in deleted_lines:
                if line=='':
                    print('oh my god')
                temp = [prj, file_name, 'deleted', commit_hash, idx, line, 0]
                commit_lines_info.append(temp)
                idx += 1


100%|██████████| 1390/1390 [00:00<00:00, 5842.99it/s]


In [6]:
train_commit_lines_df = pd.DataFrame(commit_lines_info, columns=['project', 'file', 'change type', 'commit hash', 'idx', 'code line', 'label'])

In [7]:
train_commit_lines_df.to_excel('./train_buggy_commit_lines_df.xlsx', index=False)

In [8]:
train_commit_lines_df.to_pickle('./train_buggy_commit_lines_df.pkl')

In [9]:
# buggy_commit_lines_df = pd.read_pickle('./train_buggy_commit_lines_df.pkl')

In [10]:
# buggy_commit_lines_df.info()

# <font color='red'>验证集缺陷代码行提取

In [11]:
valid_df = pd.read_pickle('../../data/jitfine/features_valid.pkl')
# buggy_changes_with_buggy_line.json中知识对buggy commit的标注
valid_df = valid_df[valid_df['is_buggy_commit']==1]
valid_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 467 entries, 0 to 5361
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   project                     467 non-null    object 
 1   parent_hashes               467 non-null    object 
 2   commit_hash                 467 non-null    object 
 3   author_name                 467 non-null    object 
 4   author_email                467 non-null    object 
 5   author_date                 467 non-null    object 
 6   author_date_unix_timestamp  467 non-null    object 
 7   commit_message              467 non-null    object 
 8   la                          467 non-null    object 
 9   ld                          467 non-null    object 
 10  fileschanged                467 non-null    object 
 11  nf                          467 non-null    object 
 12  ns                          467 non-null    object 
 13  nd                          467 no

In [12]:
path = '../../JITFine/labels for each line/buggy_changes_with_buggy_line.json'
data = open(path, encoding='utf-8')
data = json.load(data)
# data.keys()
prj_list = list(data.keys())
print(len(prj_list))
prj_list

26


['ant-ivy',
 'commons-math',
 'opennlp',
 'parquet-mr',
 'archiva',
 'deltaspike',
 'systemml',
 'commons-lang',
 'commons-net',
 'commons-collections',
 'commons-beanutils',
 'commons-codec',
 'commons-compress',
 'commons-configuration',
 'commons-digester',
 'commons-jcs',
 'commons-io',
 'commons-scxml',
 'commons-validator',
 'commons-vfs',
 'giraph',
 'jspwiki',
 'eagle',
 'commons-bcel',
 'commons-dbcp',
 'gora']

In [13]:
not_use_prj = ['archiva', 'deltaspike', 'systemml', 'jspwiki',
               'eagle']
for k in not_use_prj:
    del data[k]
prj_list = list(data.keys())
print(len(prj_list))
prj_list

21


['ant-ivy',
 'commons-math',
 'opennlp',
 'parquet-mr',
 'commons-lang',
 'commons-net',
 'commons-collections',
 'commons-beanutils',
 'commons-codec',
 'commons-compress',
 'commons-configuration',
 'commons-digester',
 'commons-jcs',
 'commons-io',
 'commons-scxml',
 'commons-validator',
 'commons-vfs',
 'giraph',
 'commons-bcel',
 'commons-dbcp',
 'gora']

In [14]:
commit_lines_info = []

for i in tqdm(range(len(valid_df))):
    commit = valid_df.iloc[i]
    prj = commit['project']
    # print(prj)
    commit_hash = commit['commit_hash']


    info = data[prj][commit_hash]['added_buggy_level']
    modified_files = list(info.keys())



    idx = 0
    for file_name in modified_files:
        file = info[file_name]

        if 'added_buggy' in file.keys():
            buggy_lines = file['added_buggy']

            for line in buggy_lines:
                temp = [prj, file_name, 'added', commit_hash, idx, line, 1]
                commit_lines_info.append(temp)
                idx += 1


        if 'added_clean' in file.keys():
            clean_lines = file['added_clean']

            for line in clean_lines:
                temp = [prj, file_name, 'added', commit_hash, idx, line, 0]
                commit_lines_info.append(temp)
                idx += 1

        if 'deleted' in file.keys():
            deleted_lines = file['deleted']

            for line in deleted_lines:
                temp = [prj, file_name, 'deleted', commit_hash, idx, line, 0]
                commit_lines_info.append(temp)
                idx += 1


100%|██████████| 467/467 [00:00<00:00, 8980.35it/s]


In [15]:
valid_commit_lines_df = pd.DataFrame(commit_lines_info, columns=['project', 'file', 'change type', 'commit hash', 'idx', 'code line', 'label'])
valid_commit_lines_df.to_excel('./valid_buggy_commit_lines_df.xlsx', index=False)


In [16]:
valid_commit_lines_df.to_pickle('./valid_buggy_commit_lines_df.pkl')


# <font color='red'>测试集缺陷代码行提取

In [17]:
test_df = pd.read_pickle('../../data/jitfine/features_test.pkl')
# buggy_changes_with_buggy_line.json中知识对buggy commit的标注
test_df = test_df[test_df['is_buggy_commit']==1]
test_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 475 entries, 0 to 5376
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   project                     475 non-null    object 
 1   parent_hashes               475 non-null    object 
 2   commit_hash                 475 non-null    object 
 3   author_name                 475 non-null    object 
 4   author_email                475 non-null    object 
 5   author_date                 475 non-null    object 
 6   author_date_unix_timestamp  475 non-null    object 
 7   commit_message              475 non-null    object 
 8   la                          475 non-null    object 
 9   ld                          475 non-null    object 
 10  fileschanged                475 non-null    object 
 11  nf                          475 non-null    object 
 12  ns                          475 non-null    object 
 13  nd                          475 no

In [18]:
path = '../../JITFine/labels for each line/buggy_changes_with_buggy_line.json'
data = open(path, encoding='utf-8')
data = json.load(data)
# data.keys()
prj_list = list(data.keys())
print(len(prj_list))
prj_list

26


['ant-ivy',
 'commons-math',
 'opennlp',
 'parquet-mr',
 'archiva',
 'deltaspike',
 'systemml',
 'commons-lang',
 'commons-net',
 'commons-collections',
 'commons-beanutils',
 'commons-codec',
 'commons-compress',
 'commons-configuration',
 'commons-digester',
 'commons-jcs',
 'commons-io',
 'commons-scxml',
 'commons-validator',
 'commons-vfs',
 'giraph',
 'jspwiki',
 'eagle',
 'commons-bcel',
 'commons-dbcp',
 'gora']

In [19]:
not_use_prj = ['archiva', 'deltaspike', 'systemml', 'jspwiki',
               'eagle']
for k in not_use_prj:
    del data[k]
prj_list = list(data.keys())
print(len(prj_list))
prj_list

21


['ant-ivy',
 'commons-math',
 'opennlp',
 'parquet-mr',
 'commons-lang',
 'commons-net',
 'commons-collections',
 'commons-beanutils',
 'commons-codec',
 'commons-compress',
 'commons-configuration',
 'commons-digester',
 'commons-jcs',
 'commons-io',
 'commons-scxml',
 'commons-validator',
 'commons-vfs',
 'giraph',
 'commons-bcel',
 'commons-dbcp',
 'gora']

In [20]:
commit_lines_info = []

for i in tqdm(range(len(test_df))):
    commit = test_df.iloc[i]
    prj = commit['project']
    # print(prj)
    commit_hash = commit['commit_hash']


    info = data[prj][commit_hash]['added_buggy_level']
    modified_files = list(info.keys())


    idx = 0
    for file_name in modified_files:
        file = info[file_name]

        if 'added_buggy' in file.keys():
            buggy_lines = file['added_buggy']

            for line in buggy_lines:
                temp = [prj, file_name, 'added', commit_hash, idx, line, 1]
                commit_lines_info.append(temp)
                idx += 1


        if 'added_clean' in file.keys():
            clean_lines = file['added_clean']

            for line in clean_lines:
                temp = [prj, file_name, 'added', commit_hash, idx, line, 0]
                commit_lines_info.append(temp)
                idx += 1

        if 'deleted' in file.keys():
            deleted_lines = file['deleted']

            for line in deleted_lines:
                temp = [prj, file_name, 'deleted', commit_hash, idx, line, 0]
                commit_lines_info.append(temp)
                idx += 1


100%|██████████| 475/475 [00:00<00:00, 9693.87it/s]


In [21]:
test_commit_lines_df = pd.DataFrame(commit_lines_info, columns=['project', 'file', 'change type', 'commit hash', 'idx', 'code line', 'label'])
test_commit_lines_df.to_excel('./test_buggy_commit_lines_df.xlsx', index=False)


In [22]:
test_commit_lines_df.to_pickle('./test_buggy_commit_lines_df.pkl')
